### Image classification with different classifiers:

Data source : https://archive.ics.uci.edu/ml/datasets/Image+Segmentation 

In [2]:
import pandas as pd
import numpy as np

#### Read input data:

In [3]:
ip_data = pd.read_csv("segmentation.data", index_col=None)

classes_series = ip_data.iloc[:, 0]
classes = classes_series.unique()
(total_samples, total_features) = ip_data.shape
total_features = total_features-1
total_classes = classes.size
class_column = 'IMAGE'

In [4]:
from sklearn import preprocessing

x = ip_data.iloc[:, 1:].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = preprocessing.scale(x) #min_max_scaler.fit_transform(x)
ip_data.iloc[:, 1:] = x_scaled

### 1) Gaussian Discriminant 

#### For each class compute the parameters using MLE :

In [5]:
class_priors = np.array(total_classes)

#### Lets model the class priors using multinouli distribution:

In [6]:
classes_counts = classes_series.value_counts()
class_priors = classes_counts/total_samples
class_priors

CEMENT       0.142857
BRICKFACE    0.142857
GRASS        0.142857
PATH         0.142857
WINDOW       0.142857
FOLIAGE      0.142857
SKY          0.142857
Name: IMAGE, dtype: float64

#### For each class, model class conditional densities p(x/y=c) with gaussian:

In [7]:
ccd_mean = ip_data.groupby([class_column]).mean() # np.array((total_classes, total_features))

In [8]:
ccd_cov = ip_data.groupby([class_column]).cov() #np.array((total_classes, total_features, total_features))

#### Now test for a sample and predict output based on max value among all classes :
As class priors are same, we can ignore this term while calculating p(y=c/x), 

y_hat = argmax p(y=c_i/x)

In [10]:
test_data = pd.read_csv("segmentation.test")

In [18]:
accurate_predictions = 0
def predict_accuracy(test_sample):
    global accurate_predictions
    test_cls = test_sample.loc[class_column]
    test_features = test_sample.drop(class_column)
    min_cls_posterior = np.inf
    predicted_cls = classes[0]
    for cls in classes:
        # Get mean and covariance for this class
        cls_mean = ccd_mean.loc[cls]
        cls_cov = ccd_cov.loc[cls]
        # take log of exponential term of MVN
        exp_term = np.dot(np.dot((test_features - cls_mean).T, np.linalg.pinv(cls_cov)), (test_features - cls_mean))
        # Instead of taking det of co-variance, take 1/det(psuedo inverse)
        det_inv_cov_term = np.absolute(np.linalg.det(np.linalg.pinv(cls_cov)))
        if det_inv_cov_term == 0:
            cls_posterior = exp_term
        else:
            cls_posterior = np.log(det_inv_cov_term) * exp_term
        if cls_posterior <= min_cls_posterior:
            min_cls_posterior = cls_posterior
            predicted_cls = cls
    
    # if predicted class is same as test sample then increment predictions count
    if predicted_cls == test_cls:
        accurate_predictions += 1

test_data.apply(predict_accuracy, axis=1)
print("Accuracy : " + str(accurate_predictions*100/test_data.shape[0]) + "%")

Accuracy : 14.285714285714286%


 ### Naive Bayes Classifier:
 
 #### Consider every feature as univariate gaussian and all are independent

In [13]:
### 1) Get mean & variance for each class and for each feature
features = ip_data.columns[1:]
features_classes_mean = ip_data.groupby([class_column]).mean().T
features_classes_var = ip_data.groupby([class_column]).var().T

In [17]:
accurate_predictions = 0
def predict_accuracy_naive_bayes(test_sample):
    global accurate_predictions
    test_cls = test_sample.loc[class_column]
    test_features = test_sample.drop(class_column)
    min_cls_posterior = np.inf
    predicted_cls = classes[0]
    for cls in classes:
        cls_posterior = 1
        for feature in features:
            fc_mean = features_classes_mean.loc[feature, cls]
            fc_var = features_classes_var.loc[feature, cls]
            if fc_var == 0:
                continue
            f_value = test_features.loc[feature]
            cls_posterior *= ((f_value-fc_mean)/fc_var) + np.log(np.sqrt(fc_var))
        
        if cls_posterior <= min_cls_posterior:
            min_cls_posterior = cls_posterior
            predicted_cls = cls
        
    if predicted_cls == test_cls:
        accurate_predictions += 1

test_data.apply(predict_accuracy_naive_bayes, axis=1)
print("Accuracy : " + str(accurate_predictions*100/test_data.shape[0]) + "%")

Accuracy : 16.476190476190474%


### Logistic regression (using liblinear package):

In [19]:
from liblinearutil import *

In [20]:
lib_ip_classes = pd.factorize(ip_data.iloc[:,0])[0]
lib_ip_features = ip_data.iloc[:, 1:]
prob = problem(lib_ip_classes, lib_ip_features.values)
param = parameter('-s 0 -B 1 -c 4') # c is set to 4 by using -C option already
m = train(prob, param)

p_labels, p_acc, p_vals = predict(pd.factorize(test_data.iloc[:,0])[0], test_data.iloc[:,1:].values, m)

Accuracy = 2.80952% (59/2100) (classification)
